In [145]:
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [146]:
from crewai import Agent, Task, Crew, Process

## Tools

In [148]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool,WebsiteSearchTool


search_tool = SerperDevTool()  
scrape_tool = ScrapeWebsiteTool() 
flight_tool = WebsiteSearchTool(
    url="https://www.booking.com",
    search_terms="flights from your city to user's destination"
)
# map_tool = MapboxTool(api_key=os.getenv("MAPBOX_API_KEY"))  # Custom Mapbox tool to display locations on map
# geocoding_tool = MapboxGeocodingTool(api_key=os.getenv("MAPBOX_API_KEY"))  # Too

## Agents

In [149]:
tourist_searcher = Agent(
    role="Tourist Place Researcher ",
    goal="Find and summarize " 
    "the top tourist places " 
    "in Pakistan " 
    "based on user interests.",
    tools=[search_tool,scrape_tool],
    verbose=True,
    backstory=(
        "A web search expert, " 
        "proficient in finding " 
        "the most exciting tourist spots " 
        "based on the user's interests."
    )
 
)

In [150]:
flight_finder = Agent(
    role="Flight Search Expert",
    goal="Find flights to top tourist spots " 
    "in Pakistan " 
    "within the user\'s budget.",
    tools=[search_tool,scrape_tool,flight_tool], 
    verbose=True,
    memory=True,
    backstory=(
        "A dedicated travel expert " 
        "skilled in finding the best flight options " 
        "within the user's budget."
    ),
)

In [151]:
hotel_finder = Agent(
    role="Hotel Search Expert",
    goal="Find hotels " 
    "near tourist destinations "
    "based on user preferences and budget.",
    tools=[search_tool,scrape_tool],
    verbose=True,
    memory=True,
    backstory=(
       "A travel expert proficient in finding " 
       "the best hotels based on location, " 
       "budget, and reviews. "
    )
 
)


In [152]:
restaurant_finder = Agent(
    role="Restaurant Search Expert",
    goal="Find the best restaurants " 
    "near tourist spots " 
    "based on user preferences (cuisine, price). ",
    tools=[search_tool,scrape_tool], 
    verbose=True,
    memory=True,
    backstory=( 
        "An expert food guide " 
        "with the ability to find restaurants " 
        "based on cuisine preferences, " 
        "ratings, and budget. "
    )
)

In [153]:
map_guide = Agent(
    role="Map Guide",
    goal="Fetch latitude and longitude" 
    "for tourist places in Pakistan.",
    tools=[search_tool,scrape_tool],  
    verbose=True,
    memory=True,
    backstory=(
        "A local guide specialized" 
        "in providing accurate geolocation data"
        "for tourist places."
    )
)

## Task

In [155]:
tourist_search_task = Task(
    description="Search the web " 
    "for top tourist places "
    "in {search_query} " 
    "based on the user's {interest}.",
    expected_output="A summary of the top tourist destinations " 
    "in {search_query} " 
    "based on the user's {interest} and {duration}.",
    agent=tourist_searcher
)


In [156]:
flight_search_task = Task(
    description="Search for flights " 
    "in {search_query} " 
    "based on user {budget}.",
    expected_output="List of {flight_search} flights " 
    "to major tourist destinations, " 
    "including airline names, " 
    "times, prices (within user's {budget}). ",
    agent=flight_finder
)

In [157]:
hotel_search_task = Task(
    description="Search for {hotel_search} " 
    "near the tourist destination " 
    "based on user preferences (location, price range, and duration). ",
    expected_output="List of {hotel_search} "
    "near the tourist destination " 
    "with ratings, prices, and booking links. ",
    agent=hotel_finder
)

In [158]:
restaurant_search_task = Task(
    description="Find {restaurant_search} "
    "near the tourist destination, " 
    "based on user preferences (cuisine, price). ",
    expected_output="List of nearby {restaurant_search} "
    "with reviews," 
    "price ranges, and cuisine type.",
    agent=restaurant_finder
)

In [159]:
geolocation_task = Task(
    description="Fetch latitude and longitude for tourist places in {search_query}.",
    expected_output="List of tourist places in {search_query} with their corresponding latitude and longitude.",
    agent=map_guide
)

## Create Crew

In [160]:
crew = Crew(
    agents=[tourist_searcher,flight_finder,hotel_finder,restaurant_finder,map_guide],
    tasks=[tourist_search_task,flight_search_task,hotel_search_task,restaurant_search_task,geolocation_task],
   verbose=True 
)

2024-09-08 12:33:26,582 - 16888 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [161]:
inputs_details={
    'search_query': 'Skardu', 
    'budget': '100000 PKR', 
    'interest': 'adventure', 
    'duration': '7 days', 
    'flight_search': 'Karachi to Skardu',  
    'hotel_search': 'Hotels in Skardu', 
    'restaurant_search': 'Restaurants in Skardu'
}



In [162]:
result = crew.kickoff(inputs=inputs_details)

 [2024-09-08 12:33:31][DEBUG]: == Working Agent: Tourist Place Researcher 
 [2024-09-08 12:33:31][INFO]: == Starting Task: Search the web for top tourist places in Skardu based on the user's adventure.


> Entering new CrewAgentExecutor chain...
I need to search for top tourist places in Skardu that are suitable for adventure activities over a span of 7 days. This will help me gather relevant information about the adventurous spots in that region.

Action: Search the internet  
Action Input: {"search_query": "top adventure tourist places in Skardu for 7 days itinerary"}   


Search results: Title: A Photo-Heavy Guide On What To Do In Skardu, Pakistan
Link: https://thesaneadventurer.com/what-to-do-in-skardu-pakistan/
Snippet: Sheosar Lake & Rama Lake. Sheosar Lake is located inside the Deosai National Park so there is no reason to miss it if visiting Deosai. Another ...
---
Title: Explore Skardu in 7 days | Skardu trip 2020 - Tripadvisor
Link: https://www.tripadvisor.com/AttractionProdu

In [163]:
combined_output = "\n\n".join([task.raw for task in result.tasks_output])


In [164]:
display(Markdown(combined_output))

**Skardu Best Adventure Tour Package 2024 (7 Days Trip)**

**Overview:**
Embark on a breathtaking 7-day adventure in Skardu, located in the heart of Gilgit-Baltistan, Pakistan. This region serves as a gateway to some of the world's highest mountain ranges, including the majestic K2. This tour is designed for adventure seekers and nature lovers, offering stunning landscapes, pristine lakes, and rich cultural heritage.

**Itinerary:**

- **Day 1: Departure from Islamabad**
  - Start your journey with a scenic drive from Islamabad to Chilas via the Karakoram Highway, with stops for photography at Naran and Babusar Pass. Overnight stay in Chilas.

- **Day 2: Chilas to Skardu**
  - Continue your journey with breathtaking views through Jaglot, where the Himalayas, Karakoram, and Hindu Kush meet. Check into your hotel in Skardu and explore the local market in the evening.

- **Day 3: Skardu City Tour and Shangrila Lake**
  - Visit Skardu Fort (Kharpocho Fort) for panoramic views, then head to Shangrila Resort (Lower Kachura Lake) for a relaxing boat ride. Overnight stay in Skardu.

- **Day 4: Upper Kachura Lake and Shigar Fort**
  - Enjoy hiking or boating at Upper Kachura Lake, followed by a visit to Shigar Fort, an ancient palace turned heritage hotel. Return to Skardu for the night.

- **Day 5: Jeep Drive to Deosai National Park**
  - Experience the stunning landscapes of Deosai National Park, known for its wildlife and breathtaking views. Visit Sheosar Lake and spend the night back in Skardu.

- **Day 6: Return to Naran**
  - Begin your return journey to Naran via Babusar Pass, enjoying the beautiful valleys. Check into a hotel in Naran and spend the evening exploring Saif-ul-Mulook Lake.

- **Day 7: Naran to Islamabad**
  - After breakfast, drive back to Islamabad, taking in the scenic views and stopping for photography along the way. Arrive in Islamabad by late afternoon.

**Cost Includes:**
- Comfortable air-conditioned transportation
- Family-friendly accommodation
- Guided sightseeing as per itinerary
- Professional local guides
- Meals (breakfast and dinner)
- Domestic flight tickets (if applicable)
- Road taxes and fuel

**Cost Excludes:**
- Meals not mentioned in the itinerary
- Visa fees and personal expenditures
- Entry fees to historical sites
- Travel and health insurance

**Why Visit Skardu?**
Skardu is known for its unique natural beauty, adventure activities, and cultural richness. It offers opportunities for hiking, boating, and exploring historical sites like Shigar Fort and Skardu Fort. The region's diverse culture reflects influences from Tibetan, Persian, and Central Asian traditions, making it a culturally enriching experience.

This itinerary provides a perfect blend of adventure and relaxation, making it an ideal travel plan for those seeking an unforgettable experience in Skardu.

For more details and booking, visit [Big Bang Tours Pakistan](https://bigbangtours.com/trip/skardu-adventure-tour-package/).

Here are the options for flights from Karachi to Skardu within your budget of 100,000 PKR:

1. **Pakistan International Airlines (PIA)**
   - **Flight**: PK-455
   - **Departure**: 06:00 AM from Karachi (KHI)
   - **Arrival**: 08:15 AM in Skardu (KDU)
   - **Duration**: 2 hours 15 minutes
   - **Price**: PKR 40,840 (one-way)

2. **Pakistan International Airlines (PIA)**
   - **Flight**: PK-455
   - **Departure**: 10:15 AM from Karachi (KHI)
   - **Arrival**: 12:30 PM in Skardu (KDU)
   - **Duration**: 2 hours 15 minutes
   - **Price**: PKR 132 (approximately PKR 40,000) (one-way)

3. **Pakistan International Airlines (PIA)**
   - **Flight**: PK-455
   - **Departure**: 12:20 PM from Karachi (KHI)
   - **Arrival**: 02:35 PM in Skardu (KDU)
   - **Duration**: 2 hours 15 minutes
   - **Price**: PKR 137 (approximately PKR 40,000) (one-way)

4. **Serene Air**
   - **Estimated lowest fare**: Approximately PKR 60,000 for a round trip.

These flights provide you with a variety of options, and all of them are within your budget. For booking and more details, you can check on platforms like Sastaticket or directly with Pakistan International Airlines. Enjoy your trip to Skardu!

Here is a list of hotels in Skardu near popular tourist destinations, along with their ratings, prices, and booking links:

1. **Hotel Mountain Lodge Holiday Home**
   - **Rating**: 5.0/5
   - **Price**: From $11 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

2. **HIKK Inn**
   - **Rating**: 5.0/5
   - **Price**: From $40 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

3. **Montagna Pods**
   - **Rating**: 5.0/5
   - **Price**: From $40 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

4. **Dewanekhas Hotel & Restaurant**
   - **Rating**: 5.0/5
   - **Price**: Approximately $30 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

5. **Arish Luxury Suites**
   - **Rating**: 5.0/5
   - **Price**: From $45 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

6. **Hispar Hotel Skardu**
   - **Rating**: 5.0/5
   - **Price**: Approximately $40 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

7. **Hotel One Skardu**
   - **Rating**: 5.0/5
   - **Price**: From $50 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

8. **Byarsa Hotel**
   - **Rating**: 5.0/5
   - **Price**: Approximately $35 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

9. **Baltistan Fort**
   - **Rating**: 5.0/5
   - **Price**: From $40 per night
   - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

10. **Des Dastaan**
    - **Rating**: 5.0/5
    - **Price**: Approximately $30 per night
    - **Booking Link**: [Tripadvisor](https://www.tripadvisor.com/Hotels-g317119-Skardu_Gilgit_Baltistan-Hotels.html)

These hotels are highly rated and located close to popular tourist attractions in Skardu, making them ideal choices for your stay. Enjoy your trip!

Here are some of the best restaurants in Skardu near popular tourist destinations, complete with their cuisine types, price ranges, and reviews:

1. **Dewanekhas Restaurant**
   - **Cuisine**: Chinese, Asian, Pakistani
   - **Price Range**: $4.00 – $20.00
   - **Rating**: 4.5/5 (55 reviews)
   - **Description**: Known for its variety in Chinese and Pakistani dishes, this restaurant also offers vegetarian and vegan options. It's highly recommended for both locals and tourists.
   - [View on Tripadvisor](https://www.tripadvisor.com/Restaurant_Review-g317119-d2654945-Reviews-Dewanekhas_Restaurant_Skardu-Skardu_Gilgit_Baltistan.html)

2. **Shahi Dewan**
   - **Cuisine**: Chinese, Bar
   - **Price Range**: Moderate
   - **Rating**: 5.0/5 (6 reviews)
   - **Description**: A great place with private dining options, offering a cozy atmosphere for enjoying Chinese cuisine.
   - [View on Tripadvisor](https://www.tripadvisor.com/Restaurant_Review-g317119-d2654945-Reviews-Dewanekhas_Restaurant_Skardu-Skardu_Gilgit_Baltistan.html)

3. **Cafe De North**
   - **Cuisine**: Italian, Chinese
   - **Price Range**: Moderate
   - **Rating**: 5.0/5 (2 reviews)
   - **Description**: A delightful spot to enjoy a mix of Italian and Chinese dishes. This cafe is known for its stunning views and atmosphere.
   - [View on Tripadvisor](https://www.tripadvisor.com/Restaurant_Review-g317119-d2654945-Reviews-Dewanekhas_Restaurant_Skardu-Skardu_Gilgit_Baltistan.html)

4. **City Inn Restaurant**
   - **Cuisine**: Middle Eastern, Pakistani
   - **Price Range**: $10.00 – $30.00
   - **Rating**: 4.5/5 (5 reviews)
   - **Description**: Celebrated as one of the best local spots for authentic Pakistani cuisine, it offers a welcoming atmosphere and delicious food.
   - [View on Tripadvisor](https://www.tripadvisor.com/Restaurant_Review-g317119-d2654945-Reviews-Dewanekhas_Restaurant_Skardu-Skardu_Gilgit_Baltistan.html)

5. **Food Street Api Xhoq Skardu**
   - **Cuisine**: Chinese, Fast Food
   - **Price Range**: Moderate
   - **Rating**: 5.0/5 (3 reviews)
   - **Description**: Offers a combination of fast food and traditional Chinese dishes in a relaxed setting, perfect for a quick bite.
   - [View on Tripadvisor](https://www.tripadvisor.com/Restaurant_Review-g317119-d2654945-Reviews-Dewanekhas_Restaurant_Skardu-Skardu_Gilgit_Baltistan.html)

6. **Hotel Summit & Restaurant**
   - **Cuisine**: Chinese, Fast Food
   - **Price Range**: Moderate
   - **Rating**: 4.5/5 (4 reviews)
   - **Description**: Known for its friendly service and variety of fast food and traditional dishes. A great spot for families.
   - [View on Tripadvisor](https://www.tripadvisor.com/Restaurant_Review-g317119-d2654945-Reviews-Dewanekhas_Restaurant_Skardu-Skardu_Gilgit_Baltistan.html)

These restaurants not only offer a variety of cuisines but are also well-reviewed by visitors, making them excellent choices for dining in Skardu while exploring the area's beauty.

Here is the list of notable tourist places in Skardu along with their corresponding latitude and longitude:

1. **Deosai National Park**  
   - Latitude: 35.0189° N  
   - Longitude: 75.4144° E  

2. **Upper Kachura Lake**  
   - Latitude: 35.3288° N  
   - Longitude: 75.5364° E  

3. **Manthokha Waterfall**  
   - Latitude: 35.0662° N  
   - Longitude: 75.9971° E  

4. **Sadpara Lake**  
   - Latitude: 35.2258° N  
   - Longitude: 75.6273° E  

5. **Kharphocho Fort (Skardu Fort)**  
   - Latitude: 35.3039° N  
   - Longitude: 75.6380° E  

6. **Katpana Desert**  
   - Latitude: 35.3139° N  
   - Longitude: 75.7530° E  

This comprehensive list provides essential geolocation data for some of the most popular tourist destinations in Skardu, Pakistan.